In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd

import datetime

from CoolProp.CoolProp import PropsSI

import matplotlib.pyplot as plt
%matplotlib inline

cwd = os.getcwd()
sys.path.append(os.path.normpath(os.path.join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(os.path.normpath(os.path.join(cwd, '..', '..', '..', 'standalone')))

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv("out.csv", index_col=0)

In [ ]:
df.head(1)

In [ ]:
start_time = datetime.datetime(month=1, day=1, year=2018, hour=0, minute=0, second=0)

In [ ]:
l = df['Simulation Time'].tolist()

In [ ]:
dt = [datetime.timedelta(seconds=x) for x in l]

In [ ]:
index = [start_time + x for x in dt]

In [ ]:
index = pd.to_datetime(index)

In [ ]:
df.set_index(index, inplace=True)

In [ ]:
df.plot(y=['GLHE Inlet Temperature [C]', 'GLHE Outlet Temperature [C]'])

In [ ]:
dT = df['GLHE Inlet Temperature [C]'].diff()
dt = df['GLHE Inlet Temperature [C]'].index.to_series().diff().dt.total_seconds()

df['dT_in/dt'] = dT/dt

In [ ]:
df.plot(y='dT_in/dt')